In [19]:
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
data_path = '../Data/cleanedData/cleaned_data.csv'
data = pd.read_csv(data_path)
area_name = 'Nam Từ Liêm'

In [21]:
area = data[data['Quận/Huyện'] == area_name]
area.shape

(37, 14)

In [22]:
month_price_path = '../Data/priceData/month_price.csv'
quarter_price_path = '../Data/priceData/quarter_price.csv'

In [23]:
month_prices = pd.read_csv(month_price_path)
quarter_prices = pd.read_csv(quarter_price_path)

month_prices.shape

(541, 26)

In [24]:
columns = 'Giá T10/22,Giá T11/22,Giá T12/22,Giá T1/23,Giá T2/23,Giá T3/23,Giá T4/23,Giá T5/23,Giá T6/23,Giá T7/23,Giá T8/23,Giá T9/23,Giá T10/23,Giá T11/23,Giá T12/23,Giá T1/24,Giá T2/24,Giá T3/24,Giá T4/24,Giá T5/24,Giá T6/24,Giá T7/24,Giá T8/24,Giá T9/24,Giá T10/24'
prices = pd.DataFrame(columns=columns.split(','))

In [25]:
quarter_to_months = {
    'Giá Q4/22': ['Giá T10/22', 'Giá T11/22', 'Giá T12/22'],
    'Giá Q1/23': ['Giá T1/23', 'Giá T2/23', 'Giá T3/23'],
    'Giá Q2/23': ['Giá T4/23', 'Giá T5/23', 'Giá T6/23'],
    'Giá Q3/23': ['Giá T7/23', 'Giá T8/23', 'Giá T9/23'],
    'Giá Q4/23': ['Giá T10/23', 'Giá T11/23', 'Giá T12/23'],
    'Giá Q1/24': ['Giá T1/24', 'Giá T2/24', 'Giá T3/24'],
    'Giá Q2/24': ['Giá T4/24', 'Giá T5/24', 'Giá T6/24'],
    'Giá Q3/24': ['Giá T7/24', 'Giá T8/24', 'Giá T9/24', 'Giá T10/24']
}


In [26]:
def change_quarter_to_month(quarter_row):
    result = pd.DataFrame(columns=columns.split(','))
    for quarter, months in quarter_to_months.items():
        if quarter in quarter_row:
            for month in months:
                result[month] = quarter_row[quarter]
    
    return result


In [27]:
for index, row in area.iterrows():
    if row['Mã lịch sử giá'] == 'M':
        matching_row = month_prices[month_prices['index'] == index].drop(columns=['index'])
        if not matching_row.empty:
            prices = pd.concat([prices, matching_row], ignore_index=True)
    if row['Mã lịch sử giá'] == 'Q':
        matching_row = quarter_prices[quarter_prices['index'] == index].drop(columns=['index'])
        if not matching_row.empty:
            prices = pd.concat([prices, change_quarter_to_month(matching_row)], ignore_index=True)

In [28]:
prices.head()

,Giá T10/22,Giá T11/22,Giá T12/22,Giá T1/23,Giá T2/23,Giá T3/23,Giá T4/23,Giá T5/23,Giá T6/23,Giá T7/23,...,Giá T1/24,Giá T2/24,Giá T3/24,Giá T4/24,Giá T5/24,Giá T6/24,Giá T7/24,Giá T8/24,Giá T9/24,Giá T10/24
0,60.0 100.0 191.5,60.0 100.0 191.5,60.0 100.0 191.5,92.3 135.7 194.4,92.3 135.7 194.4,92.3 135.7 194.4,85.7 139.2 202.8,85.7 139.2 202.8,85.7 139.2 202.8,96.0 134.4 188.9,...,85.0 155.3 207.7,85.0 155.3 207.7,85.0 155.3 207.7,89.4 152.3 206.2,89.4 152.3 206.2,89.4 152.3 206.2,54.3 160.0 263.1,54.3 160.0 263.1,54.3 160.0 263.1,54.3 160.0 263.1
1,97.0 133.0 200.0,85.0 134.0 227.0,74.0 128.0 210.0,71.0 117.0 196.0,86.0 135.0 224.0,88.0 134.0 212.0,98.0 147.0 216.0,90.0 147.0 232.0,85.0 134.0 214.0,81.0 129.0 224.0,...,98.0 129.0 184.0,90.0 131.0 200.0,112.0 158.0 218.0,125.0 180.0 251.0,125.0 171.0 250.0,120.0 175.0 258.0,131.0 175.0 250.0,134.0 179.0 250.0,142.0 194.0 265.0,148.0 207.0 284.0
2,67.0 88.0 117.0,64.0 89.0 113.0,72.0 90.0 115.0,69.0 86.0 105.0,73.0 90.0 113.0,79.0 90.0 111.0,62.0 84.0 104.0,78.0 90.0 115.0,73.0 88.0 109.0,74.0 88.0 119.0,...,80.0 94.0 119.0,76.0 96.0 133.0,85.0 104.0 129.0,88.0 108.0 143.0,96.0 117.0 151.0,97.0 118.0 151.0,95.0 118.0 153.0,103.0 124.0 162.0,99.0 128.0 180.0,106.0 136.0 192.0
3,88.0 119.0 233.0,78.0 118.0 209.0,76.0 112.0 191.0,79.0 110.0 180.0,84.0 118.0 200.0,84.0 125.0 249.0,80.0 121.0 257.0,82.0 116.0 198.0,74.0 113.0 190.0,93.0 130.0 204.0,...,100.0 137.0 202.0,100.0 133.0 220.0,112.0 152.0 264.0,124.0 169.0 257.0,129.0 175.0 281.0,132.0 176.0 264.0,139.0 171.0 225.0,145.0 174.0 226.0,145.0 185.0 259.0,150.0 185.0 247.0
4,79.7 117.2 166.7,79.7 117.2 166.7,79.7 117.2 166.7,90.3 103.0 105.1,90.3 103.0 105.1,90.3 103.0 105.1,94.2 108.0 121.1,94.2 108.0 121.1,94.2 108.0 121.1,88.9 105.4 112.7,...,107.6 123.7 128.4,107.6 123.7 128.4,107.6 123.7 128.4,123.7 133.3 143.0,123.7 133.3 143.0,123.7 133.3 143.0,121.2 133.3 154.0,121.2 133.3 154.0,121.2 133.3 154.0,121.2 133.3 154.0


In [29]:
print(prices.head())
print(prices.dtypes)


         Giá T10/22        Giá T11/22        Giá T12/22         Giá T1/23  \
0  60.0 100.0 191.5  60.0 100.0 191.5  60.0 100.0 191.5  92.3 135.7 194.4   
1  97.0 133.0 200.0  85.0 134.0 227.0  74.0 128.0 210.0  71.0 117.0 196.0   
2   67.0 88.0 117.0   64.0 89.0 113.0   72.0 90.0 115.0   69.0 86.0 105.0   
3  88.0 119.0 233.0  78.0 118.0 209.0  76.0 112.0 191.0  79.0 110.0 180.0   
4  79.7 117.2 166.7  79.7 117.2 166.7  79.7 117.2 166.7  90.3 103.0 105.1   

          Giá T2/23         Giá T3/23         Giá T4/23         Giá T5/23  \
0  92.3 135.7 194.4  92.3 135.7 194.4  85.7 139.2 202.8  85.7 139.2 202.8   
1  86.0 135.0 224.0  88.0 134.0 212.0  98.0 147.0 216.0  90.0 147.0 232.0   
2   73.0 90.0 113.0   79.0 90.0 111.0   62.0 84.0 104.0   78.0 90.0 115.0   
3  84.0 118.0 200.0  84.0 125.0 249.0  80.0 121.0 257.0  82.0 116.0 198.0   
4  90.3 103.0 105.1  90.3 103.0 105.1  94.2 108.0 121.1  94.2 108.0 121.1   

          Giá T6/23         Giá T7/23  ...          Giá T1/24  \
0  85.7 1

In [30]:
import numpy as np
import pandas as pd

# less mean max
less = []
mean = []
max = []
mean_price = pd.DataFrame(columns=prices.columns)

for col in prices.columns:
    less.clear()
    mean.clear()
    max.clear()
    for row in prices[col]:
        if isinstance(row, str):
            row_values = row.split()
            less.append(float(row_values[0]))
            mean.append(float(row_values[1]))
            max.append(float(row_values[2]))

    mean_price[col] = [np.mean(less), np.mean(mean), np.mean(max)]


In [35]:
prices.loc[1,:]

Giá T10/22     97.0 133.0 200.0
Giá T11/22     85.0 134.0 227.0
Giá T12/22     74.0 128.0 210.0
Giá T1/23      71.0 117.0 196.0
Giá T2/23      86.0 135.0 224.0
Giá T3/23      88.0 134.0 212.0
Giá T4/23      98.0 147.0 216.0
Giá T5/23      90.0 147.0 232.0
Giá T6/23      85.0 134.0 214.0
Giá T7/23      81.0 129.0 224.0
Giá T8/23      83.0 121.0 199.0
Giá T9/23      87.0 124.0 201.0
Giá T10/23     80.0 118.0 189.0
Giá T11/23     83.0 125.0 231.0
Giá T12/23     94.0 131.0 218.0
Giá T1/24      98.0 129.0 184.0
Giá T2/24      90.0 131.0 200.0
Giá T3/24     112.0 158.0 218.0
Giá T4/24     125.0 180.0 251.0
Giá T5/24     125.0 171.0 250.0
Giá T6/24     120.0 175.0 258.0
Giá T7/24     131.0 175.0 250.0
Giá T8/24     134.0 179.0 250.0
Giá T9/24     142.0 194.0 265.0
Giá T10/24    148.0 207.0 284.0
Name: 1, dtype: object

In [38]:
from model import Predictor

predictor = Predictor(mean_price.columns, mean_price.loc[1,:])

predictor.show()

AttributeError: 'Predictor' object has no attribute 'show'